# Purpose
The purpose of this notebook is to generate figure 4.

# Preliminaries

## load packages

In [4]:
%matplotlib inline
%autosave 60
%load_ext autoreload
%autoreload 2

#filesystem
import os
import glob
from pathlib import Path
from tqdm import tqdm

#plotting
import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from PIL import Image
from dateutil import rrule
import matplotlib.colors as mcolors

#data
import numpy as np
from datetime import datetime
from scipy.stats import pearsonr
import pandas as pd
import random
import ast
import pickle
import networkx as nx
import community
from datetime import datetime, timedelta
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# custom modules
from src import parameters,rfid, plot
from src import timestamps
from src import spectrogramming
from src import preprocess


Autosaving every 60 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## set paths

In [ ]:
# The ID of the DAS model that generated the segments
DAS_model = '20230219_120047'

# main project root - to reproduce, just change root to the full path to wild-mus-vocal-ecology
root = '/path/to/wild-mus-vocal-ecology'

# this directory contains jpegs
pictures_root = os.path.join(root, 'parameters','images','barn_pictures')

# this directory contains raw vocalization segments
vocal_counts_path = os.path.join(root, 'data', 'segments', 'vocal_counts')

# path to dictionary recording which boxes were recorded when and with which audiomoth
boxes_dictionary = os.path.join(root, 'parameters', 'json_files', 'boxes_recorded.json')

# paths to dictionaries for coloring plots
vocalization_colors = os.path.join(root, 'parameters', 'json_files', 'vocalization_colors.json')
seasons_colors = os.path.join(root, 'parameters', 'json_files','season_color_dict.json')
audiomoth_colors = os.path.join(root, 'parameters', 'json_files','audiomoth_colors.json')

# paths to rfid reading directories (each contains a directory for meets, stays, and box events)
all_rfid = os.path.join(root, 'data', 'rfid') # path to rfid data from 2013-2023 ("all")

# path to adult phenotypes (with sex)
adult_sexes_path = os.path.join(root, 'data', 'phenotypes', 'sexes.csv')

# timestamp (string) of umap embedding to plot in yyyymmdd_hhmmss format
umap_ID = '20230707_111132'

# paths to vocal events data
vocal_events_path = os.path.join(root,'data', 'segments', 'vocal_events' )

# paths to example spectrograms
spectrograms_path = os.path.join(root,'data', 'umap', 'spectrograms' )


# Get the data

## Vocal events

In [ ]:
#vocal events - all recordings
print('getting vocal events 2022-2023...')
v_events = pd.concat([pd.read_csv(i) for i in glob.glob(os.path.join(vocal_events_path, '*segments.csv'))])
v_events = preprocess.raw_data_from_cloud_of_mice(v_events, df_type='vocal_events')

#vocal counts - all recordings
print('getting vocal counts 2022-2023...')
v_counts = pd.concat([pd.read_csv(i) for i in glob.glob(os.path.join(vocal_counts_path, '*counts.csv'))])
v_counts = preprocess.raw_data_from_cloud_of_mice(v_counts, df_type='vocal_counts')

print('Done.')

## Organize the raw data

### Percent vocal events by season

In [ ]:
# Grouping v_events by deployment and getting the first occurrence of yearless_season for each deployment
v_events['yearless_season'] = [i.split('_')[0] for i in v_events['season']]
v_events = v_events[v_events['label'] != 'noise']
v_events_by_deployment = v_events[['deployment', 'yearless_season']].groupby('deployment').agg({'yearless_season':'first'}).reset_index()

# Counting the number of deployments for each season
num_summer_deployments = len(v_events_by_deployment[v_events_by_deployment['yearless_season'] == 'summer'])
num_winter_deployments = len(v_events_by_deployment[v_events_by_deployment['yearless_season'] == 'winter'])
num_autumn_deployments = len(v_events_by_deployment[v_events_by_deployment['yearless_season'] == 'autumn'])
num_spring_deployments = len(v_events_by_deployment[v_events_by_deployment['yearless_season'] == 'spring'])

# Printing the counts for each season
print(f'Number of summer deployments: {num_summer_deployments}')
print(f'Number of winter deployments: {num_winter_deployments}')
print(f'Number of autumn deployments: {num_autumn_deployments}')
print(f'Number of spring deployments: {num_spring_deployments}')


In [ ]:
#average detected vocalizations per box
summer_vocs = v_events[v_events['yearless_season'] == 'summer']
winter_vocs = v_events[v_events['yearless_season'] == 'winter']
autumn_vocs = v_events[v_events['yearless_season'] == 'autumn']
spring_vocs = v_events[v_events['yearless_season'] == 'spring']
total_vocs = len(v_events)
vocs_per_box = total_vocs/144
minutes_per_box = ((6594.3)*60)/144
vocs_per_minute_per_box = vocs_per_box/minutes_per_box
num_squeaks = len(v_events[v_events['label'] == 'squeak'])
num_USV = len(v_events[v_events['label'] == 'USV'])
print('There are', total_vocs, 'vocalizations, correspinding to', vocs_per_minute_per_box, "vocalizations per minute per box")
print(num_squeaks, 'of these are squeaks and', num_USV, "are USVs")
print(f"There are {num_squeaks/num_USV} times more squeaks than USVs\n\n")
print(len(summer_vocs)/total_vocs, "of vocalizations took place in summer.\n")
print(len(winter_vocs)/total_vocs, "of vocalizations took place in winter.\n")
print(len(autumn_vocs)/total_vocs, "of vocalizations took place in autumn.\n")
print(len(spring_vocs)/total_vocs, "of vocalizations took place in spring.\n")
print((len(spring_vocs)+len(summer_vocs))/total_vocs, "of vocalizations took place in spring and summer.\n")



In [6]:
# total vocalizations, squeaks, and USVs
print(f"Total vocalization: {len(v_events)}")
print(f"Total squeaks: {len(v_events[v_events['label'] == 'squeak'])}")
print(f"Total vocalization: {len(v_events[v_events['label'] == 'USV'])}")

Total vocalization: 1366171
Total squeaks: 1260452
Total vocalization: 105719


In [7]:
#average detected vocalizations per box
vocs_per_box = 1366171/144
minutes_per_box = ((6594.3)*60)/144
vocs_per_minute_per_box = vocs_per_box/minutes_per_box
print('Average vocalizations per box:', vocs_per_minute_per_box)

Average vocalizations per box: 3.4529088252986164


### Get vocalization counts by time of day

In [3]:
all_seasons = []   
for season in v_events['season'].unique():
    
    df = timestamps.get_vocalizations_by_hour_of_day(season = season, 
                                                     v_events = v_events)
    df['season'] = season
    all_seasons.append(df)
    
vocs_by_time_of_day = pd.concat(all_seasons)

### Calculate squeak--USV co-occurance

In [4]:
df = v_counts.copy()

# filter 
filtered_df = df

# get observed overlap
observed_overlap = len(filtered_df[(filtered_df['squeak_count'] > 0) & (filtered_df['USV_count'] > 0)])/len(filtered_df)

# simulate expected overlap
n_permutations = 1000  # Number of permutations
expected_overlaps = []

for _ in tqdm(range(n_permutations)):
    shuffled_squeak_count = np.random.permutation(filtered_df['squeak_count'])
    shuffled_USV_count = np.random.permutation(filtered_df['USV_count'])
    expected_overlap = len(filtered_df[(shuffled_squeak_count > 0) & (shuffled_USV_count > 0)])
    expected_overlaps.append(expected_overlap/len(filtered_df))

# compare
expected_overlaps = np.array(expected_overlaps)

p_value_right = (np.sum(expected_overlaps >= observed_overlap) + 1) / (n_permutations + 1)
p_value_left = (np.sum(expected_overlaps <= observed_overlap) + 1) / (n_permutations + 1)
p_value_two_tailed = 2 * min(p_value_right, p_value_left)

print(f'Observed overlap: {observed_overlap}')
print(f'Mean expected overlap: {np.mean(expected_overlaps)}')
print(f'P-value (right tail): {p_value_right}')
print(f'P-value (left tail): {p_value_left}')
print(f'P-value (two-tailed): {p_value_two_tailed}')



## Make the Figure

In [11]:
# save or not
save = False
path_to_save = ''

##################################################################################################
root = '/path/to/wild-mus-vocal-ecology/'
umap_df = pd.read_feather(os.path.join(root, 'data/umap/20230707_111132_UMAPembedding_withfeatures.feather'))

##################################################################################################

# plotting parameters
plt.style.use('default')
tick_label_fontsize = 6
axis_label_fontsize = 9
title_label_fontsize = 12
num_ticks = 40 # for panel F x-axis
dot_size = .5
dot_alpha = .5
voc_colormap = parameters.load_json(vocalization_colors)
season_colormap = parameters.load_json(seasons_colors)

fig, axes = plt.subplot_mosaic(mosaic="EEEEIIJJ.;"\
                                      "EEEELLMMk;"\
                                      "EEEELLMMk;"\
                                      "FFFFLLMMk;"\
                                      "FFFFLLMMk;"
                                      "FFFFLLMMk;"\
                                      "FFFFLLMMk;"\
                                      "OOOORRRR.;"\
                                      "OOOORRRR.;"\
                                      "PPPPQQQQ.;"\
                                      "PPPPQQQQ.",

                               figsize=[8.5,11], 
                               gridspec_kw={'height_ratios': [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5, 0.5, 0.5], 
                                            'width_ratios':[1,1,1,1,1,1, 1, 1, 0.1]},
                               tight_layout=True,
                               dpi=600)

##################################################################################################
### PANELS E - UMAP embedding
##################################################################################################
embedding = umap_df
sns.scatterplot(data = embedding,
                x = 'umap1',
                y = 'umap2',
                hue = embedding['label'],
                palette = voc_colormap,
                s = dot_size,
                alpha = dot_alpha, 
                ax = axes["E"], 
                linewidth = 0,
                legend = 'brief')

axes["E"].set_xlim([-2.5,12.5])
axes["E"].set_ylim([-2.5,12.5])
axes["E"].legend(frameon=False)
axes["E"].axis('off')

#################################################################################################
## PANELS F - example spectrograms - NB this requires access to raw audio
#################################################################################################

num_freq_bins = 1024
num_time_bins = 1024
spec_min_val = 1.5
spec_max_val = 4.5
nperseg = 512
fs = 192000

squeak_spec_params = {
    'fs':fs,
    'nperseg':nperseg,
    'noverlap':nperseg//4,
    'num_freq_bins':num_freq_bins,
    'num_time_bins':num_time_bins,
    'min_freq':0,
    'max_freq':96000,
    'fill_value':spec_min_val,
    'max_duration':0.25,
    'spec_min_val':spec_min_val,
    'spec_max_val':spec_max_val
}

USV_spec_params = {
    'fs':fs,
    'nperseg':nperseg,
    'noverlap':nperseg//4,
    'num_freq_bins':num_freq_bins,
    'num_time_bins':num_time_bins,
    'min_freq':0,
    'max_freq':96000,
    'fill_value':spec_min_val,
    'max_duration':0.06,
    'spec_min_val':spec_min_val,
    'spec_max_val':spec_max_val
}

# Create a grid of square images within subplot F
axes["F"].axis('off')
num_rows, num_cols = 4, 4
image_size = 0.2 
inset_axes_dict = {}
inset_ID = 1
spacing = 0.02


for row in range(num_rows):
    for col in range(num_cols):
        x_pos = (col / num_cols) + 0.05
        y_pos = 1 - (row + 1) / num_rows
        inset_axes_dict[inset_ID] = axes['F'].inset_axes([x_pos, y_pos, image_size, image_size+spacing])
        inset_ID+=1

#the example USVs 
USV_names = ['audiomoth02_20220917_031100_clip1',
            'audiomoth03_20220903_094400_clip10',
            'audiomoth03_20220818_041700_clip5',
            'audiomoth03_20230107_083200_clip27',
            'audiomoth03_20220819_091300_clip11',
            'audiomoth03_20220903_094300_clip10',
            'audiomoth03_20220904_065600_clip4',
            'audiomoth03_20230107_192400_clip1']
USV_to_plot = embedding[embedding['clip_name'].isin(USV_names)]

# plot the spectrograms and decorate the UMAP
USV_txt = [1,2,5,6,9,10,13,14]
for USV_txt, umap1, umap2 in zip(USV_txt,USV_to_plot['umap1'], USV_to_plot['umap2']):
    voc = USV_to_plot[(USV_to_plot['umap1'] == umap1) & (USV_to_plot['umap2'] == umap2)] # one vocalization
    axes["E"].scatter(voc['umap1'],voc['umap2'],c = 'black',s = dot_size+5, linewidth=0,alpha = dot_alpha) #umap dot
    axes["E"].annotate(text=USV_txt, xy=(umap1,  umap2), xytext=(0.5,0.5), textcoords = 'offset points', color='black', fontsize=tick_label_fontsize) #umap dot number
    path_to_spec = os.path.join(root, 'data', 'umap', 'spectrograms', voc['clip_name'].iloc[0]+".npy")
    spec = np.load(path_to_spec)
    inset_axes_dict[int(USV_txt)].imshow(spec, origin='lower', extent = (num_freq_bins, 0, num_time_bins, 0 ), aspect='equal')
    inset_axes_dict[int(USV_txt)].text(118, 10, USV_txt, ha="center", va="top", color="w", fontsize=axis_label_fontsize, fontname='Arial')
    inset_axes_dict[int(USV_txt)].axis('off')

#the example squeaks
squeak_names = ['audiomoth01_20220818_074400_clip54',
                 'audiomoth01_20230317_035600_clip2',
                 'audiomoth02_20230311_165300_clip18',
                 'audiomoth01_20230317_035800_clip54',
                 'audiomoth01_20221126_183000_clip13',
                 'audiomoth01_20221015_165000_clip3',
                 'audiomoth01_20220818_064400_clip4',
                 'audiomoth01_20220818_012000_clip28']
squeaks_to_plot = embedding[embedding['clip_name'].isin(squeak_names)]  

# plot the spectrograms and decorate the UMAP
squeaks_txt = [3,4,7,8,11,12,15,16]
for squeaks_txt, umap1, umap2 in zip(squeaks_txt,squeaks_to_plot['umap1'], squeaks_to_plot['umap2']):
    voc = squeaks_to_plot[(squeaks_to_plot['umap1'] == umap1) & (squeaks_to_plot['umap2'] == umap2)] # one vocalization
    axes["E"].scatter(voc['umap1'],voc['umap2'],c = 'black',s = dot_size+5, linewidth=0,alpha = dot_alpha) #umap dot
    axes["E"].annotate(text=squeaks_txt, xy=(umap1,  umap2), xytext=(0.5,0.5), textcoords = 'offset points', color='black', fontsize=tick_label_fontsize) #umap dot number
    path_to_spec = os.path.join(root, 'data', 'umap', 'spectrograms', voc['clip_name'].iloc[0]+".npy")
    spec = np.load(path_to_spec)
    inset_axes_dict[int(squeaks_txt)].imshow(spec, origin='lower', extent = (num_freq_bins, 0, num_time_bins, 0 ), aspect='equal')
    inset_axes_dict[int(squeaks_txt)].text(118, 10, squeaks_txt, ha="center", va="top", color="w", fontsize=axis_label_fontsize, fontname='Arial')
    inset_axes_dict[int(squeaks_txt)].axis('off')

##################################################################################################
### PANEL & - vocal counts by time of day
##################################################################################################

season_colormap = parameters.load_json(seasons_colors)
season_order = ['summer_22', 'autumn_22', 'winter', 'spring', 'summer_23', 'autumn_23']

# Pivot the DataFrame to get hours as columns and dates as rows
USV_heatmap_data = vocs_by_time_of_day.pivot_table(
    index=['season', 'deployment', 'moth'],  # Rows represent each recorded day
    columns=['hour'],  # Columns represent each hour interval
    values='USV_count',  # Values to fill the heatmap
    fill_value=0  # Fill missing values with 0
)
# Convert 'season' index level to a categorical type with the defined order
USV_heatmap_data.index = pd.MultiIndex.from_frame(
    USV_heatmap_data.index.to_frame().assign(
        season=pd.Categorical(USV_heatmap_data.index.get_level_values('season'), categories=season_order, ordered=True)
    ),
    names=['season', 'deployment', 'moth']
)

# Sort the DataFrame by the new categorical index
USV_heatmap_data = USV_heatmap_data.sort_index()

# Define the desired order for seasons

# Pivot the DataFrame to get hours as columns and dates as rows
squeak_heatmap_data = vocs_by_time_of_day.pivot_table(
    index=['season', 'deployment', 'moth'],  # Rows represent each recorded day
    columns=['hour'],  # Columns represent each hour interval
    values='squeak_count',  # Values to fill the heatmap
    fill_value=0  # Fill missing values with 0
)
squeak_heatmap_data.index = pd.MultiIndex.from_frame(squeak_heatmap_data.index.to_frame().assign(
        season=pd.Categorical(squeak_heatmap_data.index.get_level_values('season'), categories=season_order, ordered=True)
    ),
    names=['season', 'deployment', 'moth']
)
squeak_heatmap_data = squeak_heatmap_data.sort_index()

# Normalize the rows to the maximum value in each row
USV_heatmap_data_normalized = USV_heatmap_data.div(USV_heatmap_data.max(axis=1), axis=0).fillna(0)
squeak_heatmap_data_normalized = squeak_heatmap_data.div(squeak_heatmap_data.max(axis=1), axis=0).fillna(0)

# Aggregate data for line plots
vocs_by_time_of_day['yearless_season'] = [i.split('_')[0] for i in vocs_by_time_of_day['season'] ]
lineplot_data = vocs_by_time_of_day.groupby(['hour', 'yearless_season'])[['USV_count', 'squeak_count']].sum().reset_index()
lineplot_data['USV_count'] = lineplot_data.groupby('yearless_season')['USV_count'].transform(lambda x: x / x.max())
lineplot_data['squeak_count'] = lineplot_data.groupby('yearless_season')['squeak_count'].transform(lambda x: x / x.max())

# Plot the lineplots using seaborn
sns.lineplot(data=lineplot_data, 
             x='hour', 
             y='USV_count', 
             hue='yearless_season', 
             palette=season_colormap, 
             ax=axes["J"], legend = False)
sns.lineplot(data=lineplot_data, 
             x='hour', 
             y='squeak_count', 
             hue='yearless_season', 
             palette=season_colormap, 
             ax=axes["I"], legend = False)

# Remove x-axis labels for lineplots
axes["J"].set_xticklabels([])
axes["I"].set_xticklabels([])
axes["J"].tick_params(axis = 'y', labelsize = tick_label_fontsize)
axes["I"].tick_params(axis = 'y', labelsize = tick_label_fontsize)

# Set y-axis labels for lineplots
axes["J"].set_ylabel('')
axes["I"].set_ylabel('')
axes["J"].set_xlabel('')
axes["I"].set_xlabel('')

# Plot the heatmaps
sns.heatmap(USV_heatmap_data_normalized, cmap='YlGnBu', annot=False, cbar=False, ax=axes["M"])
axes["M"].set_title('')
axes["M"].set_xlabel('Hour', fontsize = axis_label_fontsize)
axes["M"].set_ylabel('')
axes["M"].set_yticklabels([])
axes["M"].tick_params(axis = 'x', labelsize = tick_label_fontsize)

# Plot the heatmaps
sns.heatmap(squeak_heatmap_data_normalized, cmap='YlGnBu', annot=False, cbar=False, ax=axes["L"])
axes["L"].set_title('')
axes["L"].set_xlabel('Hour', fontsize = axis_label_fontsize)
axes["L"].set_ylabel('Recorded Date', fontsize = axis_label_fontsize)
axes["L"].set_yticklabels([])
axes["L"].tick_params(axis = 'x', labelsize = tick_label_fontsize)
axes["L"].tick_params(axis = 'y', labelsize = tick_label_fontsize)

# Convert the season names to numeric codes
season_annotation_df = pd.DataFrame(columns = ['season'])
season_annotation_df['season'] = squeak_heatmap_data.reset_index()['season']
season_annotation_df_numeric = season_annotation_df.applymap(lambda x: list(season_colormap.keys()).index(x))

# Create a color list from the colormap
colors = list(season_colormap.values())
cmap = mcolors.ListedColormap(colors)
bounds = list(range(len(season_colormap) + 1))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# Plot the heatmap
sns.heatmap(
    season_annotation_df_numeric,
    cmap=cmap,
    annot=False,
    cbar=False,
    ax=axes['k'],
    linewidths=0.5,
    norm=norm
)

axes['k'].set_title('')
axes['k'].set_xlabel('')
axes['k'].set_ylabel('')
for ax in [axes['k']]:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
sns.despine(ax = axes["J"])
sns.despine(ax = axes["I"])


##################################################################################################
### PANELS O and P - minutes with vocalization by deployment
##################################################################################################

#convert the "deployment" column to a categorical data type for regression
v_counts['deployment'] = pd.Categorical(v_counts['deployment'], ordered=True)
v_counts['deployment_codes'] = v_counts['deployment'].cat.codes

#calculate the percentage of minutes with > 1 vocalization counts for each value of "deployment"
contains_squeaks = v_counts.groupby(['season', 'deployment_codes', 'moth'])['squeak_count'].apply(lambda x: (x > 0).mean() * 100).reset_index().rename(columns = {'squeak_count':'percent_minutes_with_squeaks'})
contains_USVs = v_counts.groupby(['season','deployment_codes', 'moth'])['USV_count'].apply(lambda x: (x > 0).mean() * 100).reset_index().rename(columns = {'USV_count':'percent_minutes_with_USV'})
percent_minutes_with_vocs = contains_USVs.merge(contains_squeaks, on = ['season', 'deployment_codes', 'moth'])

# plot
sns.scatterplot(data=contains_squeaks, 
            x='deployment_codes', 
            y='percent_minutes_with_squeaks', 
            s=15,
            c = contains_squeaks['season'].map(season_colormap),
            linewidth=0,
            ax=axes["O"])
sns.scatterplot(data=contains_USVs, 
            x='deployment_codes', 
            y='percent_minutes_with_USV', 
            s=15,
            c = contains_USVs['season'].map(season_colormap),
            linewidth=0,
            ax=axes["P"])

mean_squeaks = contains_squeaks.groupby('deployment_codes')['percent_minutes_with_squeaks'].median().reset_index()
axes["O"].plot(mean_squeaks['deployment_codes'], 
               mean_squeaks['percent_minutes_with_squeaks'], 
               color='grey', 
               linestyle='-', 
               linewidth=3, 
               alpha = 0.5,
               marker='o', 
               markersize = 0)

mean_USVs = contains_USVs.groupby('deployment_codes')['percent_minutes_with_USV'].median().reset_index()
axes["P"].plot(mean_USVs['deployment_codes'], 
               mean_USVs['percent_minutes_with_USV'], 
               color='grey', 
               linestyle='-', 
               alpha = 0.5,
               linewidth=3, 
               marker='o', 
               markersize = 0)

# modify plots    
deployment_dates = [i[4:] for i in sorted([i.split('-')[0] for i in v_counts['deployment'].unique()])]
for ax in ["O", "P"]:
    axes[ax].set_xticks(range(len(deployment_dates)))
    axes[ax].set_xticklabels(deployment_dates, rotation=90, fontsize=6)
    axes[ax].tick_params(axis='x', length=0)
    axes[ax].tick_params(axis='y', labelsize=6)
    sns.despine(ax=axes[ax])

    if ax == "O":
        axes[ax].set_title('Squeaks', fontsize=axis_label_fontsize)
        axes[ax].set_xlabel('')
        axes[ax].set_ylabel('Minutes with vocalization (%)', fontsize=axis_label_fontsize)
    elif ax == "P":
        axes[ax].set_title('USVs', fontsize=9)
        axes[ax].set_xlabel('recording start (mmdd)', fontsize=axis_label_fontsize)
        axes[ax].set_ylabel('', fontsize=6)

##################################################################################################
### PANEL R minutes with squeak vs USV 
##################################################################################################
df = percent_minutes_with_vocs.copy()
df['yearless_season'] = [i.split('-')[0] for i in df['season']]

correlation, p_value = pearsonr(df['percent_minutes_with_squeaks'], df['percent_minutes_with_USV'])

sns.scatterplot(data = df, 
                x = 'percent_minutes_with_squeaks', 
                y = 'percent_minutes_with_USV', 
                ax = axes["R"], 
                hue = 'yearless_season', 
                palette = season_colormap,
                s = 15, alpha = 0.75,
                legend = False)
sns.regplot(data=df, 
            x='percent_minutes_with_squeaks', 
            y='percent_minutes_with_USV', 
            scatter=False, 
            color='grey', 
            line_kws={"linestyle":"--", "linewidth":2}, 
            ax = axes["R"])

sns.despine()

axes["R"].tick_params(axis = 'both', labelsize = tick_label_fontsize)
axes["R"].set_xlabel('% minutes with squeaks', fontsize = axis_label_fontsize)
axes["R"].set_ylabel('% minutes with USVs', fontsize = axis_label_fontsize)
sns.despine()

##################################################################################################
### PANEL Q squeak vs USV co-occurance
##################################################################################################

sns.histplot(expected_overlaps, 
             kde=False, 
             bins=10, 
             ax = axes["Q"], 
             linewidth = 0, 
             color = 'grey')
axes["Q"].axvline(observed_overlap, 
                  color='red', 
                  linestyle='dashed', 
                  linewidth=1)
axes["Q"].set_xlabel('Minutes with squeaks and USVs co-occurring (% total)', fontsize = axis_label_fontsize)
axes["Q"].set_ylabel('Frequency', fontsize = axis_label_fontsize)
axes["Q"].text(observed_overlap + axes["Q"].get_xlim()[1] * 0.005, 
               axes["Q"].get_ylim()[1] * 0.9, 
               'Actual Overlap\n(p < 0.001)', 
               fontsize=tick_label_fontsize, 
               color='red', 
               ha='left')

axes["Q"].text(np.mean(expected_overlaps) + axes["Q"].get_xlim()[1] * 0.005, 
               axes["Q"].get_ylim()[1] * 0.9, 
               'Expected Overlaps\n(1,000 permutations)', 
               fontsize=tick_label_fontsize, 
               color='grey', 
               ha='left')

axes["Q"].tick_params(axis = 'both', labelsize = tick_label_fontsize)
axes["Q"].set_xlim([0,0.03])
sns.despine()

#save
if save:
    plt.savefig(os.path.join(path_to_save, 'Figure4.svg'))
    plt.savefig(os.path.join(path_to_save, 'Figure4.jpeg'), dpi = 600)


## Statistics

In [28]:
import statsmodels.stats.multicomp as multi
import statsmodels.stats.multitest as smm
from statsmodels.formula.api import ols
from docx import Document
from docx.shared import Inches
from src import modelselection
import scipy.stats as stats

In [34]:
statistics_dir = ''
save = False # if true, save statistics csvs and output to resubmission_statistics


### Panel C
minutes with squeaks ~ season

In [5]:
data = percent_minutes_with_vocs
dependent_var = 'percent_minutes_with_squeaks'
independent_var = 'yearless_season'
data['yearless_season'] = [i.split('_')[0] for i in data['season']]
independent_var = 'yearless_season'
data = data.dropna(subset = dependent_var)
analysis_name = 'figure4_panelC'

order = ['summer', 'autumn', 'winter', 'spring']
sns.boxplot(data = data, 
            x = 'yearless_season', 
            y = dependent_var, 
            order = order)

# Fit the one-way ANOVA model
formula = dependent_var + f" ~ C({independent_var})"
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model)
print(anova_table)

# Perform Tukey's HSD post-hoc test
mc = multi.MultiComparison(data[dependent_var], data[independent_var])
tukey_result = mc.tukeyhsd()
tukey_result = pd.DataFrame(data=tukey_result.summary().data[1:], columns=tukey_result.summary().data[0])

# Print the results of the Tukey HSD test
print(tukey_result)

if save:
    
    #save the analysis results
    modelselection.save_ANOVA_tables_to_word(analysis_name+': '+formula, anova_table, tukey_result, os.path.join(statistics_dir, analysis_name +'.docx'))
    
    #save the data the analysis was run on
    data.to_csv(os.path.join(statistics_dir, analysis_name + '.csv'), index = False)

### Panel D
Percent minutes with USV ~ season

In [6]:
data = percent_minutes_with_vocs
dependent_var = 'percent_minutes_with_USV'
independent_var = 'yearless_season'

data['yearless_season'] = [i.split('_')[0] for i in data['season']]
independent_var = 'yearless_season'
data = data.dropna(subset = dependent_var)
analysis_name = 'figure4_panelD'

order = ['summer', 'autumn', 'winter', 'spring']
sns.boxplot(data = data, 
            x = 'yearless_season', 
            y = dependent_var, 
            order = order)

# Fit the one-way ANOVA model
formula = dependent_var + f" ~ C({independent_var})"
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model)
print(anova_table)

# Perform Tukey's HSD post-hoc test
mc = multi.MultiComparison(data[dependent_var], data[independent_var])
tukey_result = mc.tukeyhsd()
tukey_result = pd.DataFrame(data=tukey_result.summary().data[1:], columns=tukey_result.summary().data[0])

# Print the results of the Tukey HSD test
print(tukey_result)

if save:
    
    #save the analysis results
    modelselection.save_ANOVA_tables_to_word(analysis_name+': '+formula, anova_table, tukey_result, os.path.join(statistics_dir, analysis_name +'.docx'))
    
    #save the data the analysis was run on
    data.to_csv(os.path.join(statistics_dir, analysis_name + '.csv'), index = False)

### Panel E
vocalizations ~ (time of day) * (season)

In [7]:
# squeaks
# Define the bins and labels 
# 1 = early morning (midnight to 6am)
# 2 = morning (6am to noon)
# 3 = afternoon (noon to 6pm)
# 4 = night
bins = [0, 6, 12, 18, 24]
labels = [1, 2, 3, 4]

# Use pd.cut to create the hour_interval column
vocs_by_time_of_day['hour_interval'] = pd.cut(vocs_by_time_of_day['hour'], bins=bins, labels=labels, right=True)
vocs_by_time_of_day['yearless_season'] = [i.split('_')[0] for i in vocs_by_time_of_day['season']]
dependent_var = 'squeak_count'
data = vocs_by_time_of_day.dropna(subset = [dependent_var])


for season in sorted(data['yearless_season'].unique()):
    print(season)
    analysis_name = 'figure4_panelE_squeak'+'_'+season
    this_season  = data[data['yearless_season'] == season] 
    formula = dependent_var + ' ~ C(hour_interval)'
    model = ols(formula, data=this_season).fit()
    anova_table = sm.stats.anova_lm(model)
    print(anova_table)
    p_value = anova_table['PR(>F)']['C(hour_interval)']
    
    if p_value < (0.05)/4:
        
        # Combine season and hour into a single factor
        this_season['group'] = this_season['yearless_season'].astype(str) + '-' + this_season['hour_interval'].astype(str)

        # Perform Tukey's HSD post-hoc test
        mc = multi.MultiComparison(this_season[dependent_var], this_season['group'])
        tukey_result = mc.tukeyhsd()
        tukey_df = pd.DataFrame(data=tukey_result.summary().data[1:], columns=tukey_result.summary().data[0])

        # Filter to just show comparisons within hours
        def is_same_hour(row):
            group1_hour = row['group1'].split('-')[1]
            group2_hour = row['group2'].split('-')[1]
            return group1_hour == group2_hour

        # Filter to just show comparisons within seasons
        def is_same_season(row):
            group1_season = row['group1'].split('-')[0]
            group2_season = row['group2'].split('-')[0]
            return group1_season == group2_season

        tukey_result = tukey_df[tukey_df.apply(is_same_season, axis=1)]
        print(tukey_result)
        
    else:
        
        tukey_result = pd.DataFrame()
        print('not significant\n')


    if save:

        #save the analysis results
        modelselection.save_ANOVA_tables_to_word(analysis_name+': '+formula, anova_table, tukey_result, os.path.join(statistics_dir, analysis_name +'.docx'))

        #save the data the analysis was run on
        data.to_csv(os.path.join(statistics_dir, analysis_name + '.csv'), index = False)

In [8]:
# USV
# Define the bins and labels 
# 1 = early morning (midnight to 6am)
# 2 = morning (6am to noon)
# 3 = afternoon (noon to 6pm)
# 4 = night (6pm to midnight)
bins = [0, 6, 12, 18, 24]
labels = [1, 2, 3, 4]

# Use pd.cut to create the hour_interval column
vocs_by_time_of_day['hour_interval'] = pd.cut(vocs_by_time_of_day['hour'], bins=bins, labels=labels, right=True)
vocs_by_time_of_day['yearless_season'] = [i.split('_')[0] for i in vocs_by_time_of_day['season']]
dependent_var = 'USV_count'
data = vocs_by_time_of_day.dropna(subset = [dependent_var])
analysis_name = 'figure4_panelE_USV'

for season in sorted(data['yearless_season'].unique()):
    print(season)
    analysis_name = 'figure4_panelE_USV'+'_'+season
    this_season  = data[data['yearless_season'] == season] 
    formula = dependent_var + ' ~ C(hour_interval)*C(hour_interval)'
    model = ols(formula, data=this_season).fit()
    anova_table = sm.stats.anova_lm(model)
    print(anova_table)
    p_value = anova_table['PR(>F)']['C(hour_interval)']
    
    if p_value < (0.05)/4: # bonferroni correction because were doing an ANOVA for each season
        
        # Combine season and hour into a single factor
        this_season['group'] = this_season['yearless_season'].astype(str) + '-' + this_season['hour_interval'].astype(str)

        # Perform Tukey's HSD post-hoc test
        mc = multi.MultiComparison(this_season[dependent_var], this_season['group'])
        tukey_result = mc.tukeyhsd()
        tukey_df = pd.DataFrame(data=tukey_result.summary().data[1:], columns=tukey_result.summary().data[0])

        # Filter to just show comparisons within hours
        def is_same_hour(row):
            group1_hour = row['group1'].split('-')[1]
            group2_hour = row['group2'].split('-')[1]
            return group1_hour == group2_hour

        # Filter to just show comparisons within seasons
        def is_same_season(row):
            group1_season = row['group1'].split('-')[0]
            group2_season = row['group2'].split('-')[0]
            return group1_season == group2_season

        tukey_result = tukey_df[tukey_df.apply(is_same_season, axis=1)]
        print(anova_table)
        print(tukey_result)
        
        sns.boxplot(data = this_season, x = 'hour_interval', y = 'USV_count')
        
    else:
        
        tukey_result = pd.DataFrame()
        print('not significant')

    if save:

        #save the analysis results
        modelselection.save_ANOVA_tables_to_word(analysis_name+': '+formula, anova_table, tukey_result, os.path.join(statistics_dir, analysis_name +'.docx'))

        #save the data the analysis was run on
        data.to_csv(os.path.join(statistics_dir, analysis_name + '.csv'), index = False)

### Panel F
Correlation between squaks and USVs

In [9]:
from scipy.stats import pearsonr

data = percent_minutes_with_vocs

# Calculate Pearson's correlation coefficient and p-value
correlation, p_value = pearsonr(data['percent_minutes_with_squeaks'], data['percent_minutes_with_USV'])

print(f'Pearson correlation coefficient: {correlation}')
print(f'R^2: {correlation**2}')
print(f'P-value: {p_value}')

sns.scatterplot(data = data, x = 'percent_minutes_with_squeaks', y = 'percent_minutes_with_USV')
sns.regplot(data=data, x='percent_minutes_with_squeaks', y='percent_minutes_with_USV', scatter=False, color='grey', line_kws={"linestyle":"--"})
sns.despine()



### Panel G
Squeak and USV co-occurance

In [10]:
# Step 0
df = v_counts

# Step 1: Filter the DataFrame
#filtered_df = df[(df['squeak_count'] > 0) | (df['USV_count'] > 0)]
filtered_df = df

# Step 2: Calculate Observed Overlap
observed_overlap = len(filtered_df[(filtered_df['squeak_count'] > 0) & (filtered_df['USV_count'] > 0)])/len(filtered_df)

# Step 3: Simulate Expected Overlap
n_permutations = 1000  # Number of permutations
expected_overlaps = []

for _ in tqdm(range(n_permutations)):
    shuffled_squeak_count = np.random.permutation(filtered_df['squeak_count'])
    shuffled_USV_count = np.random.permutation(filtered_df['USV_count'])
    expected_overlap = len(filtered_df[(shuffled_squeak_count > 0) & (shuffled_USV_count > 0)])
    expected_overlaps.append(expected_overlap/len(filtered_df))

# Step 4: Compare Observed to Expected
expected_overlaps = np.array(expected_overlaps)
mean_expected_overlap = np.mean(expected_overlaps)
std_expected_overlap = np.std(expected_overlaps)

p_value_right = (np.sum(expected_overlaps >= observed_overlap) + 1) / (n_permutations + 1)
p_value_left = (np.sum(expected_overlaps <= observed_overlap) + 1) / (n_permutations + 1)
p_value_two_tailed = 2 * min(p_value_right, p_value_left)
t_stat = (observed_overlap - np.mean(expected_overlaps)) / std_expected_overlap

sns.histplot(expected_overlaps, kde=False, bins=30)
plt.axvline(observed_overlap, color='red', linestyle='dashed', linewidth=2)
plt.xlabel('Overlapping Minutes (% Expected)')
plt.ylabel('Frequency')
plt.title('Distribution of Expected Overlaps')
plt.show()

print(f'Observed overlap: {observed_overlap}')
print(f'Mean expected overlap: {np.mean(expected_overlaps)}')
print(f'P-value (right tail): {p_value_right}')
print(f'P-value (left tail): {p_value_left}')
print(f'P-value (two-tailed): {p_value_two_tailed}')
print(f't-stat: {t_stat}')

